In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
from kneed import KneeLocator
from sklearn.neighbors import NearestNeighbors
from numpy import inf

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

import pandas_datareader.data as web
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from pandas import ExcelWriter
from pandas import ExcelFile
import os
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from matplotlib.pyplot import figure

import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, adjusted_rand_score, silhouette_samples
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler,MaxAbsScaler
from sklearn.metrics import silhouette_score
import random

from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from kneed import KneeLocator

from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA

from sklearn.datasets import make_blobs
from sklearn.cluster import OPTICS
from sklearn.cluster import DBSCAN
import numpy as np
import matplotlib.pyplot as plt
import itertools

COLOR_BOLD = "\033[95m"
COLOR_END = "\033[0m"

In [2]:
def loop_files(string_to_find):
    '''
    Returns list of file paths containing certain string
    '''
    filepaths = []
    for root, dirs, files in os.walk('Export/'):
        for name in files:
            if name.endswith('.csv'):

                filepath = os.path.join(root, name)
                if f'{string_to_find}' in name:
                    filepaths.append(filepath)
    return filepaths

In [3]:
def read_returns(file_path):
    movements = pd.read_csv(f'{file_path}',
                              header=0,
                              index_col=0,
                              parse_dates=True)
    
    return movements

In [4]:

def find_knee(movements, min_samples):
    '''
    Finds knee for data set. AKA the epsilon, or distance
    points need to be from centroid to be considered a cluster
    '''
    
    nearest_neighbors = NearestNeighbors(n_neighbors=min_samples)
    neighbors = nearest_neighbors.fit(movements)
    distances, indices = neighbors.kneighbors(movements)

    distances = np.sort(distances[:,1], axis=0)



    i = np.arange(len(distances))
    knee = KneeLocator(i, distances,
                       S=200, curve='convex',
                       direction='increasing',
                       interp_method='polynomial')
    '''
    fig = plt.figure(figsize=(5, 3))

    knee.plot_knee()

    plt.xlabel("Points")
    plt.ylabel("Distance")
    '''
    return distances[knee.knee]

In [5]:
def tune_n_clusters(movements, upper_bound = 21, random_state = 10, init = 'k-means++'):
    
    '''
    returns optimal cluster number
    '''
    
    n_clusters = list(range(2,upper_bound))

    sil_dict = {}

    for cluster in n_clusters:
        clusterer = KMeans(n_clusters = cluster, random_state=random_state, init=init)
        cluster_labels = clusterer.fit(movements)

        sil_avg = silhouette_score(movements, clusterer.labels_)

        sil_dict[cluster] = sil_avg
        
    #return max(sil_dict, key = sil_dict.get)
    return sil_dict


In [28]:
def get_n_clusters(movements, random_state=10, max_iter=1000):
    '''
    Solves for n_clusters progrimatically using
    SSE elbow method, and Shilouette method.
    
    Uses both random init and k-means++ init.
    '''
    n_clusters = []
    #------
    kmeans_kwargs = {
        "init": 'k-means++',
        "max_iter": max_iter,
        "random_state": random_state,
    }

    # A list holds the SSE values for each k
    sse = []
    for k in range(1, 15):
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(movements)
        sse.append(kmeans.inertia_)

    kl = KneeLocator(range(1, 15), sse, curve="convex", direction="decreasing")

    n_clusters.append(kl.elbow)
    #------
    kmeans_kwargs = {
        "init": "random",
        "max_iter": 1000,
        "random_state": random_state,
    }

    # A list holds the SSE values for each k
    sse = []
    for k in range(1, 15):
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(movements)
        sse.append(kmeans.inertia_)

    kl = KneeLocator(range(1, 15), sse, curve="convex", direction="decreasing")

    n_clusters.append(kl.elbow)
    #------
    kmeans_kwargs = {
        "init": "k-means++",
        "max_iter": 1000,
        "random_state": random_state,
    }

    # A list holds the silhouette coefficients for each k
    silhouette_coefficients = []

    # Notice you start at 2 clusters for silhouette coefficient
    for k in range(2, 15):
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(movements)
        score = silhouette_score(movements, kmeans.labels_)
        silhouette_coefficients.append(score)
        
    #return len(silhouette_coefficients), len(list(range(2, 15)))
    zipped = pd.DataFrame({
        '0': list(range(2, 15)),
        '1': silhouette_coefficients
    })

    n_clusters.append(int(zipped.loc[zipped['1'].idxmax()][0]))

    kmeans_kwargs = {
        "init": "random",
        "max_iter": 1000,
        "random_state": 42,
    }

    # A list holds the silhouette coefficients for each k
    silhouette_coefficients = []

    # Notice you start at 2 clusters for silhouette coefficient
    for k in range(2, 15):
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(movements)
        score = silhouette_score(movements, kmeans.labels_)
        silhouette_coefficients.append(score)

    zipped = pd.DataFrame({
        '0': list(range(2, 15)),
        '1': silhouette_coefficients
    })
    
    n_clusters.append(int(zipped.loc[zipped['1'].idxmax()][0]))
    #------

    return list(set(n_clusters))

In [7]:
def opti_cluster(movements, eps, upper_bound = 50, DBS = True):
    '''
    Returns number of cluster for certain number of min_samples
    '''
    labels_dict = {}
    ms = list(range(1,upper_bound))
    
    if DBS:
        for i in ms:
            db = DBSCAN(eps = eps, min_samples=i)
            labels = db.fit_predict(movements)


            labels = set([label for label in labels if label >= 0])
            labels_dict[i] = len(labels)
        return labels_dict
        
    if not DBS:
        for i in ms:
            db = OPTICS(eps = eps, min_samples=i)
            labels = db.fit_predict(movements)


            labels = set([label for label in labels if label >= 0])

            labels_dict[i] = len(labels)
        return labels_dict


In [8]:
def KMEANS_DS(movements,
              n_clusters=3,
              init='k-means++',
              n_init=50,
              max_iter=1000,
              random_state=10,
              algorithm='lloyd',
              PCA_components = 3,
              scaler = MinMaxScaler()):
    '''
    Returns tuple of Silhoutte Score and Predicted Labels
    '''
    preprocessor = Pipeline([
        ("scaler", scaler),
        ("pca", PCA(n_components=PCA_components, random_state=random_state)),
    ])

    clusterer = Pipeline([
        (
            "kmeans",
            KMeans(
                n_clusters=n_clusters,
                init=init,
                n_init=n_init,
                max_iter=max_iter,
                random_state=random_state,
            ),
        ),
    ])

    pipe = Pipeline([("preprocessor", preprocessor), ("clusterer", clusterer)])

    fitted_movements = pipe.fit(movements)

    labels = pipe.predict(movements)

    preprocessed_data = pipe["preprocessor"].transform(movements)

    predicted_labels = pd.DataFrame(index=range(nrow),
                                    columns=['ticker', 'group'])

    i = 0
    for ticker in list(movements.index):
        current_prediction = fitted_movements.predict(
            [list(movements.iloc[i])])[0]
        predicted_labels.loc[i, 'ticker'] = ticker
        predicted_labels.loc[i, 'group'] = current_prediction
        i = i + 1

    sil_score = silhouette_score(preprocessed_data, predicted_labels['group'])

    return sil_score, predicted_labels

In [9]:
#DBSCAN Params List
eps = []
min_samples = []
metric = [
    'cityblock', 'cosine', 'euclidean', 'l1', 'l2', 'manhattan', 'braycurtis',
    'canberra', 'chebyshev', 'correlation', 'dice', 'hamming', 'jaccard',
    'kulsinski', 'mahalanobis', 'minkowski', 'rogerstanimoto', 'russellrao',
    'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule'
]
n_jobs = [-1]

In [10]:
def DBSCAN_DS(movements,
              eps=0.5,
              min_samples=5,
              metric='euclidean',
              metric_params=None,
              algorithm='auto',
              leaf_size=30,
              p=None,
              n_jobs=None,
              **kwargs):

    preprocessor = Pipeline([
        ("scaler", scaler),
        ("pca", PCA(n_components=PCA_components, random_state=10)),
    ])

    clusterer = Pipeline([
        (
            "DBSCAN",
            DBSCAN(
                eps=eps,
                min_samples=min_samples,
                metric=metric,
                algorithm=algorithm,
                p=2,
                leaf_size=leaf_size,
            ),
        ),
    ])

    pipe = Pipeline([("preprocessor", preprocessor), ("clusterer", clusterer)])

    labels = pipe.fit_predict(movements)

    #preprocessed_data = pipe["preprocessor"].transform(movements)
    d = {'ticker': movements.index.values, 'group': labels}
    predicted_labels = pd.DataFrame(data=d,
                                    index=range(nrow),
                                    columns=['ticker', 'group'])

    return predicted_labels

In [11]:
def OPTICS_DS(movements,
              scaler=MinMaxScaler(),
              PCA_components=3,
              random_state=10,
              min_samples=5,
              max_eps=inf,
              metric='minkowski',
              p=2,
              metric_params=None,
              cluster_method='xi',
              eps=None,
              xi=0.05,
              predecessor_correction=True,
              min_cluster_size=None,
              algorithm='auto',
              leaf_size=30,
             **kwargs):

    preprocessor = Pipeline([
        ("scaler", scaler),
        ("pca", PCA(n_components=PCA_components, random_state=10)),
    ])

    clusterer = Pipeline([
        (
            "OPTICS",
            OPTICS(
                min_samples=5,
                max_eps=inf,
                metric='minkowski',
                p=2,
                metric_params=None,
                cluster_method='xi',
                eps=eps,
                xi=0.05,
                predecessor_correction=True,
                min_cluster_size=None,
                algorithm='auto',
                leaf_size=30,
                memory=None,
                n_jobs=None,
            ),
        ),
    ])

    pipe = Pipeline([("preprocessor", preprocessor), ("clusterer", clusterer)])

    labels = pipe.fit_predict(movements)

    #preprocessed_data = pipe["preprocessor"].transform(movements)
    d = {'ticker': movements.index.values, 'group': labels}
    predicted_labels = pd.DataFrame(data=d,
                                    index=range(nrow),
                                    columns=['ticker', 'group'])

    return predicted_labels

In [12]:
def SUMMARYSTATS_DS(predicted_labels, sharpes_file_name, returns_file_name):
    '''
    Returns Constituent Companies of Portfolio and their return
    in during covid in excess of SP500 avg return during same time frame
    '''
    #Portfolio Construction
    sharpes = pd.read_csv(f"{sharpes_file_name}",
                          header=0,
                          index_col=0,
                          parse_dates=True)

    merged = sharpes.merge(predicted_labels, on=['ticker'])

    max_sharpes = merged.groupby('group')['sharpes', 'ticker'].max()

    # Testing Portfolio Returns During Covid

    movements = pd.read_csv(f'{returns_file_name}',
                            header=0,
                            index_col=0,
                            parse_dates=True)

    portfolio = movements[movements.index.isin(max_sharpes['ticker'])]

    portfolio = portfolio.T

    overall = (portfolio.iloc[:, :] + 1).prod() - 1
    average = overall.mean()

    #Excess Return


    excess = average - spExcess


    return portfolio.columns, excess

# Test

In [13]:
# import historical s&p 500 data as data frame
sp500_hist = pd.read_csv('sp500_historical.csv',
                          header=0,
                          index_col=0,
                          parse_dates=True)

sp500_hist = sp500_hist['2020-02-03':'2020-04-14']

spExcess = (sp500_hist['Adj Close'].pct_change()+1).prod()-1

In [14]:
sp500_hist

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-02-03,3235.659912,3268.439941,3235.659912,3248.919922,3248.919922,3757910000
2020-02-04,3280.610107,3306.919922,3280.610107,3297.590088,3297.590088,3995320000
2020-02-05,3324.909912,3337.580078,3313.750000,3334.689941,3334.689941,4117730000
2020-02-06,3344.919922,3347.959961,3334.389893,3345.780029,3345.780029,3868370000
2020-02-07,3335.540039,3341.419922,3322.120117,3327.709961,3327.709961,3730650000
2020-02-10,3318.280029,3352.260010,3317.770020,3352.090088,3352.090088,3450350000
2020-02-11,3365.870117,3375.629883,3352.719971,3357.750000,3357.750000,3760550000
2020-02-12,3370.500000,3381.469971,3369.719971,3379.449951,3379.449951,3926380000
2020-02-13,3365.899902,3385.090088,3360.520020,3373.939941,3373.939941,3498240000


In [15]:
spExcess

-0.12399808941797585

In [16]:
sp500_OCO = (sp500_hist['Close'] - sp500_hist['Open'])/ sp500_hist['Open']

In [17]:
(sp500_OCO+1).prod()-1

-0.02031609205109486

In [ ]:
get_n_clusters(mo)

In [40]:
###### KMEANS Params List
n_clusters = [2,3,4,6]
pca_components = [2,3]
scaler = [MinMaxScaler(), MaxAbsScaler()]
init = ['k-means++', 'random']

In [52]:
ret_files = ['Export/ac_detrend_ret.csv']

In [48]:
sharpe_files= ['Export/sharpes_monthly_diff_OCO.csv']#loop_files('monthly')

In [29]:
clusters_per_file ={}
for file in ret_files:
    movements = read_returns(file)
    clusters = get_n_clusters(movements)
    
    clusters_per_file[file] = clusters

In [34]:
clusters_per_file

{'Export/ac_detrend_ret.csv': [2, 3, 4, 6],
 'Export/oco_detrend_ret.csv': [9, 2, 7]}

In [18]:
sharpe_files

['Export/sharpes_monthly_diff_OCO.csv',
 'Export/sharpes_monthly_diff_PC.csv',
 'Export/sharpes_monthly_OCO.csv',
 'Export/sharpes_monthly_PC.csv']

In [53]:
#iter_list[378] #best kmeans combo for this iterlist
iter_list = list(itertools.product(n_clusters, #0
                                   pca_components, #1
                                   scaler, #2
                                   init, #3
                                   ret_files, #4
                                   sharpe_files)) #5

In [54]:
len(iter_list)

8

In [89]:
test = [(11,
 3,
 MinMaxScaler(),
 'k-means++',
 'Export/ac_detrend_ret.csv',
 'Export/sharpes_monthly_diff_OCO.csv')]

In [55]:
sil_master = []
predicted_labels_master = []
portfolio_companies_master = []
excess_master = []


for combo in iter_list:
    movements = read_returns(combo[4])

    nrow = movements.shape[0]
    ncol = movements.shape[1]

    sil_score, predicted_labels = KMEANS_DS(
        movements,
        n_clusters=combo[0],
        init=combo[3],
        scaler = combo[2],
        PCA_components = combo[1]
    )

    portfolio_companies, excess = SUMMARYSTATS_DS(
        predicted_labels, combo[5], 'returns_covid_PC.csv')
    
    sil_master.append(sil_score)
    predicted_labels_master.append(predicted_labels)
    portfolio_companies_master.append(portfolio_companies)
    excess_master.append(excess)

In [56]:
max(excess_master)

-0.1036465927024153

In [57]:
excess_master.index(max(excess_master))

6

In [58]:
iter_list[6]

(6,
 3,
 MinMaxScaler(),
 'k-means++',
 'Export/ac_detrend_ret.csv',
 'Export/sharpes_monthly_diff_OCO.csv')

In [96]:
iter_list[48]

(11,
 3,
 MinMaxScaler(),
 'k-means++',
 'Export/ac_detrend_ret.csv',
 'Export/sharpes_monthly_diff_OCO.csv')

In [84]:
x = read_returns('returns_covid.csv')
y =read_returns('returns_covid_OCO.csv')
z =read_returns('returns_covid_PC.csv')

In [85]:
(x.T+1).prod()-1

A      -0.045179
AAPL   -0.072566
ABC     0.045115
ABT     0.022952
ADBE   -0.029532
          ...   
XEL    -0.042492
XOM    -0.316967
XRX    -0.465842
YUM    -0.244682
ZION   -0.350627
Length: 226, dtype: float64

In [86]:
(y.T+1).prod()-1

A       0.267706
AAPL    0.235499
ABC     0.307878
ABT     0.125392
ADBE    0.236242
          ...   
XEL     0.218855
XOM    -0.342609
XRX    -0.256696
YUM    -0.042408
ZION   -0.119034
Length: 226, dtype: float64

In [87]:
(z.T+1).prod()-1

A      -0.042749
AAPL   -0.070365
ABC     0.049717
ABT     0.027259
ADBE   -0.029532
          ...   
XEL    -0.036290
XOM    -0.307161
XRX    -0.465842
YUM    -0.241297
ZION   -0.345821
Length: 226, dtype: float64

In [ ]:
n_clusters = get_n_clusters(movements)

In [11]:
#OPTICS param lists
scaler = [MinMaxScaler(), MaxAbsScaler()]
PCA_components = [2, 3, 4, 5]
min_samples = [2, 3, 4, 5, 6, 7, 8]
max_eps = [2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5]
metric = [
    'minkowski', 'manhattan'
]
cluster_method = ['xi', 'dbscan']
eps = []
min_cluster_size = [1,2,3,4,5,6,7,8,9]
algorithm = ['auto']
n_jobs = [-1]

In [ ]:
iter_DB = list(itertoools.product())

In [ ]:
eps_per_ret_file = []
for file in ret_files:
    movements = read_returns(file)
    pca_movements = PCA.fit_transfrom(n_components = 3)
    EPS = find_knee()

In [ ]:
pca_movements = ()

In [ ]:
eps = find_kneeKnee()

In [ ]:
for combo in iter_list:
    